In [1]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

In [2]:

"""Functions to preproccess the image, they do not need to be defined because they are a part of PIL, but I included them here."""
#load the image
def load_image(image_path):
    image = Image.open(image_path)
    #print(f"Image loaded with size: {image.size}")
    return image

#turn the image into an array for processing
def image_to_array(image):
    image_array = np.array(image)
    return image_array

#normalize image in range 0-1 (can help if we want to use fourier transform)
def normalize_image(array):
    normalized_array = array / 255.0
    return normalized_array

#plot image
def visualize_image(array, title="Image"):
    plt.imshow(array)
    plt.title(title)
    plt.axis('off')
    plt.show()


#main function
def preprocess_image(image_path):
    # Load image
    image = load_image(image_path)

    # Convert to array
    array = image_to_array(image)

    # Normalize the image
    normalized_array = normalize_image(array)

    # Visualize the image (optional)
    #visualize_image(array, "Original Image")

    return normalized_array


#name of the picture file goes here
#output of processed image is an array
processed_image = preprocess_image("sample.jpg")


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\jdrga\\iCloudDrive\\ UVA\\Third Semester\\ECE 2700\\Final Project\\sample.jpg'

In [ ]:
#check contents (displays a large array of values ranging from 0-1)
print(processed_image)

[[[1.         1.         1.        ]
  [1.         1.         1.        ]
  [1.         1.         1.        ]
  ...
  [0.90588235 0.92941176 0.91372549]
  [0.91372549 0.95294118 0.94901961]
  [0.78823529 0.81960784 0.82745098]]

 [[1.         1.         1.        ]
  [1.         1.         1.        ]
  [1.         1.         1.        ]
  ...
  [0.81176471 0.83529412 0.82745098]
  [0.92156863 0.96078431 0.95686275]
  [0.89019608 0.91372549 0.91372549]]

 [[1.         1.         1.        ]
  [1.         1.         1.        ]
  [1.         1.         1.        ]
  ...
  [0.7372549  0.74509804 0.73333333]
  [0.92156863 0.9372549  0.94117647]
  [0.86666667 0.88235294 0.87843137]]

 ...

 [[0.03137255 0.05882353 0.09019608]
  [0.04313725 0.07843137 0.11372549]
  [0.03921569 0.0745098  0.10980392]
  ...
  [0.16862745 0.34509804 0.18431373]
  [0.19215686 0.36862745 0.21568627]
  [0.19215686 0.36862745 0.21568627]]

 [[0.05490196 0.07058824 0.11372549]
  [0.0745098  0.09803922 0.14509804]


In [ ]:

"""there are predefined functions for denormalize_image, array_to_image, save_image, etc, but i added them above so we can debug if we need to"""
def save_image(image, save_path):
    image.save(save_path)
    #print(f"Image saved to {save_path}")


def denormalize_image(normalized_array):
    # Check the range of the input array
    if np.min(normalized_array) < 0:
        # Scale [-1, 1] to [0, 255]
        denormalized_array = ((normalized_array + 1) * 127.5).astype(np.uint8)
    else:
        # Scale [0, 1] to [0, 255]
        denormalized_array = (normalized_array * 255).astype(np.uint8)

    return denormalized_array

def array_to_image(array):
    #2D = GrayScale
    if array.ndim == 2:
        return Image.fromarray(array, mode="L")  # Grayscale
    #3D = RGB
    elif array.ndim == 3 and array.shape[2] == 3:
        return Image.fromarray(array, mode="RGB")  # RGB



# Function to recreate the image
def recreate_image(normalized_array, save_path=None):
    # Denormalize the array
    denormalized_array = denormalize_image(normalized_array)

    # Convert back to an image
    recreated_image = array_to_image(denormalized_array)

    # Save the image if a save path is provided
    if save_path:
        save_image(recreated_image, save_path)

    # Visualize the recreated image (optional)
    #visualize_image(denormalized_array, "Recreated Image")

    return recreated_image

processed_image = preprocess_image("sample.jpg")

#to visualize image either uncomment the visualize_image function or look through the save_path
recreated_image = recreate_image(processed_image, save_path="recreated_image.jpg")
